# May 21, 2024: display all modes of a model in a single grid

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 

from scipy import sparse, stats
import glob
from tqdm import tqdm
import ants
from nipype.interfaces import afni
from itertools import product, combinations

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT

from matplotlib import colors
from palettable.tableau import Tableau_20

# # ignore user warnings
# import warnings
# warnings.filterwarnings("ignore") #, category=UserWarning)

240617-13:07:55,418 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.6.1


In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

# gt.seed_rng(args.SEED)
np.random.seed(args.SEED)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

args.unit = 'seswise' # runwise/seswise/subwise : unit sample of dataset
args.denst = 15 # density of fc matrix

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results'
ROI_RESULTS_path = f'{ROI_path}/{DESC}/{args.unit}/correlation_graph/density-{args.denst}'
FC_path = f'{ROI_RESULTS_path}/corr_mats'
SBM_path = f'{ROI_RESULTS_path}/sbms'
NPY_path = f'{ROI_RESULTS_path}/npy'
os.system(f'mkdir -p {NPY_path}')
NII_path = f'{ROI_RESULTS_path}/niis'
os.system(f'mkdir -p {NII_path}/indiv')
os.system(f'mkdir -p {NII_path}/group')
HTML_path = f'{ROI_RESULTS_path}/htmls'
os.system(f'mkdir -p {HTML_path}/group')
os.system(f'mkdir -p {HTML_path}/group/indexes')
# sbm model class
args.dc = 'dc'
args.sbm = 'h' 

In [4]:
def combine_htmls(args, SBM, html_files):
  # Create the main HTML content
  html_content = """
  <!DOCTYPE html>
  <html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>HTML Grid with Headings</title>
    <style>
      body {
        display: grid;
        grid-template-columns: repeat(auto-fill, minmax(1000px, 1fr));
        gap: 20px;
        padding: 20px;
      }
      iframe {
        width: 1000px;
        height: 1000px;
        border: none;
      }
      .grid-item {
        text-align: center;
        margin-bottom: 10px;
      }
      h2 {
        margin: 0;
      }
    </style>
  </head>
  <body>
  """
  # Add iframes and headings for each HTML file
  for html_file in html_files:
    # Extract filename (without extension) for heading
    filename, _ = os.path.splitext(html_file.split('/')[-1])
    heading = f'<h2>{filename}</h2>'  # Capitalize the filename for title
    iframe_tag = f'<div class="grid-item">{heading}<iframe src="{html_file}"></iframe></div>'
    html_content += iframe_tag
  # Close the HTML content
  html_content += """
  </body>
  </html>
  """
  # Write the content to an index.html file
  with open(os.path.join(f'{HTML_path}/group/indexes', f"{SBM}_desc-index.html"), "w") as f:
      f.write(html_content)
  print("index.html has been created successfully.")
  
if args.sbm in ['a', 'd']:
  SBM = f'sbm-{args.dc}-{args.sbm}'
  html_files = sorted(glob.glob(f'{HTML_path}/group/*{SBM}*', recursive=True))
  combine_htmls(args, SBM, html_files)
elif args.sbm in ['h']:
  SBM = f'sbm-{args.dc}-{args.sbm}'
  for level in np.arange(3):
    html_files = sorted(glob.glob(f'{HTML_path}/group/*{SBM}*level-{level}*', recursive=True))
    combine_htmls(args, f'{SBM}_level-{level}', html_files)

index.html has been created successfully.
index.html has been created successfully.
index.html has been created successfully.
